## this is for who will be cheking this lab.
+ i abandoned this notebook 

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
categorical = pd.read_csv("categorical.csv")
numerical = pd.read_csv("numerical.csv")
target = pd.read_csv("target.csv")

In [3]:
categorical

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,other,27,H,M,3,L,G,C,C,2,...,0,2,96,2,96,2,96,2,96,2
95408,TX,24,H,M,3,L,F,A,C,1,...,50,1,96,3,96,3,96,3,96,3
95409,MI,30,H,M,3,L,E,B,C,3,...,38,1,96,3,95,1,96,10,94,10
95410,CA,24,H,F,2,L,F,A,C,1,...,40,5,90,11,96,8,97,1,86,12


In [4]:
numerical

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,0,25.0,25.0,25.0,9,25.000000,184568,0,1,12
95408,1,48.000000,7,9,1,0,31,43,19,4,...,0,20.0,20.0,20.0,9,20.000000,122706,1,1,2
95409,1,60.000000,5,9,0,0,18,46,20,7,...,4,3.0,10.0,10.0,3,8.285714,189641,1,3,34
95410,0,58.000000,7,9,0,0,28,35,20,9,...,18,5.0,21.0,18.0,4,12.146341,4693,1,4,11


In [5]:
data = pd.concat([numerical,categorical],axis = 1)
data1 = pd.concat([numerical,categorical,target],axis = 1)

In [6]:
# ok so lets go with the upscaling
target["TARGET_B"].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [7]:
X = data
y = target["TARGET_B"]

In [8]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) # default fraction is .2
X_train_imb = pd.DataFrame(X_train) # u only balance the train data.
trainset = pd.concat((X_train, y_train), axis=1)


In [9]:
# oversample the data
category_0 = trainset[trainset['TARGET_B'] == 0]
category_1 = trainset[trainset['TARGET_B'] == 1]

category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))
train_upsampled = pd.concat([category_1_oversampled, category_0], axis=0)

In [10]:
train_upsampled

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B
20113,0,76.000000,5,9,0,0,43,19,52,5,...,12,89,7,95,7,95,12,86,10,1
19484,0,74.000000,5,4,22,3,26,74,8,12,...,1,95,12,95,9,96,10,88,4,1
66156,1,59.000000,5,9,0,0,29,26,44,4,...,1,94,1,95,3,95,5,94,1,1
35127,1,83.000000,1,0,3,0,5,35,19,0,...,1,93,2,95,6,95,10,93,2,1
58869,1,46.000000,2,3,15,0,30,28,36,7,...,1,95,10,92,12,95,10,90,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,0,94.000000,5,9,0,0,33,21,33,13,...,2,87,1,90,8,95,7,87,1,0
21243,0,70.000000,3,9,5,2,37,10,32,26,...,8,95,12,94,10,97,2,86,12,0
45891,2,66.000000,3,6,0,0,36,11,62,4,...,1,96,2,96,2,96,2,96,2,0
43567,1,76.000000,5,3,1,0,59,22,31,3,...,7,87,11,95,12,96,3,86,9,0


In [11]:
#so the upsampled data is in order ,so better to mix it just in case we do a cross validation in the future.
train_upsampled = train_upsampled.sample(frac=1)
train_upsampled["TARGET_B"].value_counts()

1    67970
0    67970
Name: TARGET_B, dtype: int64

In [12]:
# i will just normalize all my data, i prefer to do it just in case
X_train = train_upsampled.drop('TARGET_B',axis = 1)
y_train = train_upsampled['TARGET_B']
len(X_train.columns)

337

In [13]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.shape

(135940, 337)

In [14]:
#splitting into categorical and numerical
X_train_num = X_train.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(include = object)

X_test_num = X_test.select_dtypes(include = np.number)
X_test_cat = X_test.select_dtypes(include = object)

In [15]:
# normalize numericals
transformer = MinMaxScaler().fit(X_train_num) 
numericals_train_scaled = transformer.transform(X_train_num)
numericals_test_scaled = transformer.transform(X_test_num)

In [16]:
# Convert numericals back to DataFrame to retain the index
X_test_num_df = pd.DataFrame(numericals_test_scaled, index=X_test_num.index, columns=X_test_num.columns)

In [17]:
# encode categoricals
encoder = OneHotEncoder(drop='first').fit(X_train_cat)
categoricals_train_encoded = encoder.transform(X_train_cat).toarray()
categoricals_test_encoded = encoder.transform(X_test_cat).toarray()

In [18]:
# Convert categoricals back to DataFrame
X_test_cat_df = pd.DataFrame(categoricals_test_encoded, index=X_test_cat.index)

In [19]:
# Concatenate numerical and categorical dataframes
X_test_df = pd.concat([X_test_num_df, X_test_cat_df], axis=1)

In [20]:
X_train = np.concatenate((numericals_train_scaled,categoricals_train_encoded),axis=1)
X_test = np.concatenate((numericals_test_scaled,categoricals_test_encoded),axis=1)

In [21]:
# and we are ready to go with the random forest model

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

class_model = clf.fit(X_train, y_train)

0.6244078269824923
0.6070934473651113


0    22599
1     1254
Name: TARGET_B, dtype: int64

array([[13802,  8797],
       [  575,   679]], dtype=int64)

In [22]:
print("precision: ",precision_score(y_test,y_pred))    
print("recall: ",recall_score(y_test,y_pred))  # i think that in this case we prefer higher recall  
print("f1: ",f1_score(y_test,y_pred))

precision:  0.07165470662726889
recall:  0.5414673046251993
f1:  0.1265610438024231


In [23]:
# ok so we have some decent scores already ,but we want to make it better with feature selection ,im curious about PCA so i will use that one.
pca = PCA(n_components=70)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [24]:
# now run the randomforest model again to see the changes in the results
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train_pca, y_train)
print(clf.score(X_train_pca, y_train))
print(clf.score(X_test_pca, y_test))

y_pred = clf.predict(X_test_pca)
predicted_donors = pd.DataFrame({
    'Predicted_TARGET_B': y_pred
}, index=X_test_df.index)

display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))



0.637384140061792
0.6227308933886723


0    22599
1     1254
Name: TARGET_B, dtype: int64

array([[14199,  8400],
       [  599,   655]], dtype=int64)

In [25]:

# Create a boolean mask for donors
donors_mask = (y_pred == 1)


In [26]:
predicted_donors_pca_df = pd.DataFrame(X_test_pca[donors_mask], columns=[f'PC{i+1}' for i in range(X_test_pca.shape[1])])


In [27]:
# Add the TARGET_B prediction to the DataFrame
predicted_donors_pca_df['Predicted_TARGET_B'] = 1  # Since donors_mask only includes predicted donors


In [28]:
predicted_donors_pca_df['TARGET_D'] = target.loc[predicted_donors_pca_df.index, 'TARGET_D']

In [32]:
predicted_donors_pca_df.to_csv('final_predicted_donors.csv', index=False)

In [37]:
predicted_donors_pca_df["Predicted_TARGET_B"].value_counts()

1    9055
Name: Predicted_TARGET_B, dtype: int64

# well not too much of a change tbh
+ true negatives 14102 non donors were correctly identified
+ false positives 8497 actual non donors were incorrectly predicted as donors
+ false negatives 597 actual donors were missed and predicted as non donors
+ true positives 657 actual donors were correctly predicted

+ *False Positives*: In a donor solicitation scenario, false positives represent wasted resources since they indicate non-donors who were targeted as potential donors. The cost here includes marketing materials, postage, and time spent on outreach.
+ *False Negatives*: False negatives represent lost donation opportunities. These are individuals who would have donated but were not solicited by the organization.

### what i would do is cost sensitive learning ,implement training where the model is adjusted to minimize the more costly type of error (false negatives in this case)

# Here starts the last step


In [38]:
# ok lets go do the regression model on only the df with only donors
X = predicted_donors_pca_df.drop(columns=['Predicted_TARGET_B', 'TARGET_D'])
y = predicted_donors_pca_df['TARGET_D']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [44]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse,r2

(12.35767485194086, -0.02145765703570035)